In [ ]:
# Creem la conex amb la db MYSQL
import mysql.connector

db = mysql.connector.connect(
    host="containers-us-west-11.railway.app",
    port="5513",
    username="root",
    password="lBAHrHt8GHcznpsaqfRw",
    database="railway"
)

In [ ]:
colors = {
    'R1': 'bg-cyan-600',
    'R3': 'bg-red-600',
    'R4': 'bg-orange-600',
    'R11': 'bg-sky-600',
    'R12': 'bg-yellow-300',
}

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Llegim el excel i el guardem en un dataframe
df = pd.read_excel('./Horaris.xlsx')

In [ ]:
# Creem un cursor per poder fer les consultes
cursor = db.cursor()

In [ ]:
# Extraiem el ID_PARADA del dataframe i el guardem en una llista
parades = df['ID_PARADA'].unique()

parades

print(parades)

# Inserim les parades a la taula PARADES
for parada in parades:
    # Comprovem que no existeixi la parada
    print(parada)
    cursor.execute("SELECT * FROM stations WHERE name = %s", (parada,))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim la parada
    sql = "INSERT INTO stations (name) VALUES (%s)"
    val = (parada,)
    cursor.execute(sql, val)


# Guardem els canvis
db.commit()

In [ ]:
# Canviem del df cada nom de parada pel seu id
for _, fila in df.iterrows():
    sql = "SELECT id FROM stations WHERE name = %s"
    val = (fila['ID_PARADA'],)
    cursor.execute(sql, val)
    result = cursor.fetchone()
    # Canviem de la fila el nom de la parada pel seu id
    try:
        df.loc[df['ID_PARADA'] == fila['ID_PARADA'], 'ID_PARADA'] = result[0]
        print("fila: ", fila)
    except:
        print("error en la fila: ", fila)

df

In [ ]:
# A tota la columna HORA la passem a string i posteriorment li treiem els ultims 3 caracters
df['HORA'] = df['HORA'].astype(str)
df['HORA'] = df['HORA'].str[:-3]

In [ ]:
# Creem la linia a la taula linies si no existeix
for _, fila in df.iterrows():
    # Comprovem que no existeixi la linia
    cursor.execute("SELECT * FROM routes WHERE name = %s", (fila['LINIA'],))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim la linia
    sql = "INSERT INTO routes (name, colour) VALUES (%s, %s)"
    val = (fila['LINIA'], colors[fila['LINIA']])
    cursor.execute(sql, val)

In [ ]:
# Inserim les dades de l'excel a la db
for _, row in df.iterrows():
    # Abans que res comprovem que no existeixi el registre
    cursor.execute("SELECT * FROM schedules WHERE station_id = %s AND time = %s AND train_num = %s AND stop_number = %s", (row['ID_PARADA'], row['HORA'], row['ID_VIAJE'], row['ORDEN']))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim el registre
    sql = "INSERT INTO schedules (route_id, station_id, time, train_num, stop_number) VALUES (%s, %s, %s, %s, %s)"
    val = (row['LINIA'], row['ID_PARADA'], row['HORA'], row['ID_VIAJE'], row['ORDEN'])
    try:
            cursor.execute(sql, val)
            db.commit()
    except Exception as e:
        print(e)
